[View in Colaboratory](https://colab.research.google.com/github/david7joy/RNNProjects/blob/master/Baby_Name_Generator.ipynb)

In [2]:
ls /content/drive/ColabNotebooks/ProjectRNN/

Baby Name Generator.ipynb  babynames.txt  dinos.txt  logs/  setting drive.ipynb


In [311]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
tf.reset_default_graph()

In [0]:
import numpy as np
from tensorflow.contrib import rnn
import random
import os
import time

In [0]:
logs_path = '/content/drive/ColabNotebooks/ProjectRNN/logs/'
writer = tf.summary.FileWriter(logs_path)

In [332]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
ls /drive

ls: cannot access '/drive': No such file or directory


In [0]:
os.chdir('/content/drive/ColabNotebooks/ProjectRNN/')

In [359]:
file = os.listdir()
file = file[2]
file

'dinos.txt'

In [0]:
with open(file) as f:
    training_data = f.read()
    training_data = training_data.replace('\n',' ')
    training_data = training_data.lower()

In [0]:
chars = list(set(training_data))

In [362]:
data_size,vocab_size = len(training_data),len(chars)
print("The size of the data is %d and vocab size %d"%(data_size,vocab_size))

The size of the data is 19909 and vocab size 27


In [0]:
char_to_ix = {ch:i for i,ch in enumerate(sorted(chars))}
ix_to_char = {i:ch for i,ch in enumerate(sorted(chars))}

In [0]:
# no of hidden cells or units in rnn
n_hidden = 128

# hyperparameters 
learning_rate = .0005
training_iters = 150000
display_step = 1000
time_steps = 3
num_input = 1
batch_size = 128

In [0]:
x = tf.placeholder("float",[batch_size,time_steps,num_input])  # the size of each input is 1*19
y = tf.placeholder("float",[batch_size,vocab_size])

In [0]:
weights = {'out':tf.Variable(tf.random_normal([n_hidden,vocab_size]))}
biases = {'out':tf.Variable(tf.random_normal([vocab_size]))}

In [0]:
def RNN(x,weights,biases):
    x = tf.reshape(x,[-1,time_steps])
    x = tf.split(x,time_steps,1)
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])
    #rnn_cell = rnn.BasicLSTMCell(n_hidden)
    outputs,states = rnn.static_rnn(rnn_cell,x,dtype=tf.float32)
    check1,check2 = outputs,states
    return tf.matmul(outputs[-1],weights['out']) + biases['out']

In [0]:
pred = RNN(x,weights,biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
grads_and_vars = optimizer.compute_gradients(cost)
capped_gvs = [(tf.clip_by_value(grad, -2, 2), var) for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

In [0]:
# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [395]:
#initializing the variable

init = tf.global_variables_initializer()

start_time = time.time()
with tf.Session() as sess:
    sess.run(init)
    total_loss = 0
    total_acc  = 0
    offset = 0
    r1 = offset
    r2 = offset+(batch_size*time_steps)
    
    #saver = tf.train.Saver()
    for steps in range(1, training_iters + 1):
        if offset > 16200:
            offset = 0
            r1=offset
            r2 = offset+(batch_size*time_steps)
            

        #getting the input sequence
        #print("steps :%d offset1: %d range: %d" %(steps,offset,batch_size*time_steps))
        #print(r1,r2,steps)
        X = [char_to_ix[str(training_data[i])] for i in range(r1, r2)]
        X = np.reshape(np.array(X),[batch_size,time_steps,1])
        batch_size = X.shape[0]
        r1 = r2
        r2 = r2+(batch_size*time_steps)

        # getting ready the True Y
        zeros_array = np.zeros((batch_size,vocab_size), dtype=float)
        for i in range(batch_size):
            zeros_array[i][char_to_ix[str(training_data[offset + time_steps])]] = 1.0
            offset += time_steps
            
        Y = np.reshape(zeros_array, [batch_size, -1])
        
        #_, acc, cost,one_hot_pred = sess.run([optimizer, accuracy, cost, pred], feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        _, acc, loss, onehot_pred = sess.run([training_op, accuracy, cost, pred], feed_dict={x: X, y: Y})
        total_loss += loss
        total_acc += acc
        if steps % display_step == 0:
            print(steps,(100*total_acc)/steps,total_loss/steps)
            

    # Save the variables to disk.
    #save_path = saver.save(sess, "/Users/davidjoy/Desktop/Github:Study Material/6341 - Applied Machine Learning/Rnn/Dinosaur Project")
    #print("Model saved in path: %s" % save_path)
    
    #sampling
    print("Total Mins {}".format((time.time()-start_time)/60))
    while True:
      inputwords = "enter 3 letters " 
      enteredletters = input(inputwords)
      letters = enteredletters.strip()
      letters = list(letters)
      if len(letters) != time_steps:
            continue
      try:
        letters = [char_to_ix[i] for i in letters]
        for i in range(10):
          fixing_shape = np.zeros((batch_size,3,1), dtype=float)
          cell_input = np.reshape(np.array(letters), [-1, time_steps, 1])
          cell_input = cell_input+fixing_shape
          pred_letter = sess.run(pred,feed_dict={x:cell_input})
          pred_letter = np.reshape(np.array(pred_letter[1]),(1,vocab_size))
          pred_location = int(tf.argmax(pred_letter,axis=1).eval())
          pred_location
          enteredletters += "%s"%(ix_to_char[pred_location])
          letters.append(pred_location)
          letters = letters[1:]
          names = enteredletters.split()
          for i in names:
            print(i)
      except:
        print("letter not in dictionary")

1000 33.49921875 2.235990055322647
2000 44.0046875 1.8079196329712868
3000 50.21536458333333 1.5633492973645529
4000 54.17578125 1.4131377943605186
5000 56.8325 1.314915013563633
6000 58.86080729166667 1.239735170563062
7000 60.36305803571429 1.184519168147019
8000 61.57177734375 1.140108802817762
9000 62.571354166666666 1.1030431514845953
10000 63.38546875 1.0727496922343969
11000 64.06569602272727 1.0477741292091933
12000 64.43776041666666 1.0330513514305155
13000 64.8578125 1.0163571087970182
14000 65.32745535714285 0.998990242251328
15000 65.76520833333333 0.9828368821044763
16000 66.153173828125 0.9688453332539648
17000 66.53005514705882 0.9553194113549064
18000 66.82391493055556 0.9445675472799274
19000 67.03782894736842 0.9365434409474072
20000 67.2756640625 0.9279617691472173
21000 67.50513392857142 0.9195682346324126
22000 67.73224431818181 0.9116462865133177
23000 67.8976222826087 0.9056232815397822
24000 68.08600260416667 0.8992147284336388
25000 68.2046875 0.895041388880014

70000 69.19291294642858 0.8594771341924157
71000 69.21465669014084 0.85909782306787
72000 69.2266818576389 0.8591195967598921
73000 69.22144691780822 0.8599572996003171
74000 69.20935388513513 0.860764743396962
75000 69.180125 0.8621405942006906
76000 69.13942228618421 0.8637790140372358
77000 69.08564326298702 0.8656536698918064
78000 69.0412860576923 0.8670486147705561
79000 69.00471716772152 0.8681801304307919
80000 68.972314453125 0.8694064283449203
81000 68.94155092592592 0.8705383106915303
82000 68.91983612804879 0.8714967506869538
83000 68.90106362951808 0.8721558928223978
84000 68.88148251488096 0.8729100232734567
85000 68.87409007352942 0.8731203264643165
86000 68.87370094476744 0.8732695244176443
87000 68.87078843390805 0.8735934688359841
88000 68.86490589488636 0.874080075792291
89000 68.84212605337079 0.8754074052324455
90000 68.812734375 0.8768718375954363
91000 68.77953296703296 0.8784924454453227
92000 68.74821671195652 0.8799624407058176
93000 68.71909442204301 0.881567

138000 67.641015625 0.9401799836122039
139000 67.62533723021583 0.9414602291796276
140000 67.61048549107143 0.9427052484782679
141000 67.59775044326241 0.9438822722517429
142000 67.57944542253522 0.9456738625468922
143000 67.55907998251749 0.9476443078607529
144000 67.53181423611112 0.9499333381212006
145000 67.50592672413794 0.9517700063921254
146000 67.48322452910959 0.9533570029749854
147000 67.45948129251701 0.9550728307433274
148000 67.4406566722973 0.9564156125499993
149000 67.4196990352349 0.9579570326527093
150000 67.39704166666667 0.959582185506622
Total Mins 21.767718601226807
enter 3 letters rop
rope
ropel
ropelt
ropelts
ropelts
ropelts
s
ropelts
sa
ropelts
sau
ropelts
saur
ropelts
sauru
enter 3 letters tro
tros
trosa
trosau
trosaur
trosauru
trosaurus
trosaurus
trosaurus
c
trosaurus
ch
trosaurus
chi


KeyboardInterrupt: ignored